In [121]:
# libraries

import numpy as np
from utils import *
from geomutils import *
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

In [122]:
# apposto

def errorAndJacobian(x, P, Z):
    
    M = np.zeros((3,9))
    M[0, :3] = P[:3]
    M[1,3:6] = P[:3]
    M[2, 6:] = P[:3]

    h = np.dot(M, x[:9]) + x[9:] # 3x9 * 9x1 + 3x1 = 3x1    
    
    e = h - Z

    J = np.zeros((3,12))
    J[:3, :9] = M
    J[:3, 9:] = np.eye(3)    

    return e, J

In [126]:
def doICP(x_guess, points, Z):        
    x = x_guess.copy()
    H = np.zeros((12,12))
    b = np.zeros((12))        

    for j in range(points.shape[0]):
        e, J = errorAndJacobian(x, points[j,:], Z[j,:])
        H += J.T @ J
        b += J.T @ e

    dx = - np.linalg.solve(H, b.T)
    #print("dx\n", dx)
    x += dx.T    

    A = np.concatenate((x[:3], x[3:6], x[6:9]))
    
    A = A.reshape((3,3))
    #print("R\n", A)

    u, s, vh = np.linalg.svd(A, full_matrices=True)

    # if s is a diagonal matrix with 1s on the diagonal, then the matrix R is a rotation matrix

    res = np.allclose(A, np.dot(u*s, vh))
    #print("res", res)

    R = np.dot(u, vh)    
    t = x[9:]

    return R, t

In [129]:
## data

n_points = 5

# Generate random points in 3d
points = np.random.random((n_points, 3))

# ideal position of the world in respect to the robot, in our case the HMD
x_true = np.array([0, 0, 0, np.pi/2, np.pi/6, np.pi])
X_true = v2t(x_true)
print("X_true\n", X_true)
#print("X_True\n", X_true)

# test with good initial guess
x_gg = np.array([0.5, 0.5, 0.5, 0.1, 0.1, 0.1])

x_guess = x_true + x_gg
#print("x_guess\n", x_guess)
X_guess = v2t(x_guess)
#print("X_guess\n", X_guess)

R_guess = X_guess[:3, :3]

guess = np.reshape(R_guess, (9))
guess = np.append(guess, X_guess[:3, 3])
# correct guess

n_points = points.shape[0]
P_world_hom = np.ones((n_points, 4))
P_world_hom[:, :3] = points
#print("P_world_hom\n", P_world_hom)
Z_hom = np.dot(X_true, P_world_hom.T)
#print("Z_hom\n", Z_hom)
Z = Z_hom[:3, :].T

print(Z)

X_true
 [[-8.66025404e-01 -1.06057524e-16  5.00000000e-01  0.00000000e+00]
 [-5.00000000e-01 -1.22464680e-16 -8.66025404e-01  0.00000000e+00]
 [ 1.53080850e-16 -1.00000000e+00  5.30287619e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 0.05834117 -1.06205698 -0.69895365]
 [-0.49983643 -0.53372943 -0.78112834]
 [ 0.18535673 -0.41924053 -0.46267826]
 [-0.01119928 -0.88418789 -0.75305574]
 [-0.79340953 -0.52496877 -0.22559516]]


In [130]:
### CORE ###
R, t = doICP(guess, points, Z)

## print results
print("Ground truth\n", X_true)

print("R\n", R)
print("t\n", t)


Ground truth
 [[-8.66025404e-01 -1.06057524e-16  5.00000000e-01  0.00000000e+00]
 [-5.00000000e-01 -1.22464680e-16 -8.66025404e-01  0.00000000e+00]
 [ 1.53080850e-16 -1.00000000e+00  5.30287619e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
R
 [[-8.66025404e-01 -7.33014689e-16  5.00000000e-01]
 [-5.00000000e-01  3.10830784e-16 -8.66025404e-01]
 [ 4.71844785e-16 -1.00000000e+00 -9.67669439e-16]]
t
 [-1.11022302e-16 -8.88178420e-16 -1.11022302e-16]
